In [46]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
import argparse

In [47]:
def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=250, type=int)
    parser.add_argument("--number-of-clauses", default=20000, type=int)
    parser.add_argument("--T", default=25000, type=int)
    parser.add_argument("--s", default=10.0, type=float)
    parser.add_argument("--depth", default=1, type=int)
    parser.add_argument("--hypervector-size", default=128, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=256, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument("--max-included-literals", default=32, type=int)

    args = parser.parse_args()
    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    return args



In [48]:

def getConnections(size, index):
        x = index % size
        y = index // size
       
        connections = []
        if y > 0:
            connections.append(x+(y-1)*size)
            if x < size-1:
                connections.append(x+1+(y-1)*size)
        if x > 0:
            connections.append(x-1+y*size)
        if x < size-1:
            connections.append(x+1+y*size)
        if y < size-1:
            connections.append(x+(y+1)*size)
            if x > 0:
                connections.append(x-1+ (y+1)*size)
        return connections

In [49]:
# read the data from data/hex_games_1_000_000_size_7.csv
data = np.genfromtxt('data/hex_games_1_000_000_size_7.csv', delimiter=',', dtype=np.int32, skip_header=1)
data

collum_names =['cell0_0', 'cell0_1', 'cell0_2', 'cell0_3', 'cell0_4', 'cell0_5', 'cell0_6', 'cell1_0', 'cell1_1', 'cell1_2', 'cell1_3', 'cell1_4', 'cell1_5', 'cell1_6', 'cell2_0', 'cell2_1', 'cell2_2', 'cell2_3', 'cell2_4', 'cell2_5', 'cell2_6', 'cell3_0', 'cell3_1', 'cell3_2', 'cell3_3', 'cell3_4', 'cell3_5', 'cell3_6', 'cell4_0', 'cell4_1', 'cell4_2', 'cell4_3', 'cell4_4', 'cell4_5', 'cell4_6', 'cell5_0', 'cell5_1', 'cell5_2', 'cell5_3', 'cell5_4', 'cell5_5', 'cell5_6', 'cell6_0', 'cell6_1', 'cell6_2', 'cell6_3', 'cell6_4', 'cell6_5', 'cell6_6']


In [50]:
data.shape


(1000000, 50)

In [51]:
getConnections(7, 15)


[8, 9, 14, 16, 22, 21]

In [52]:
# separate the last column from the rest of the data
X_data = data[:, :-1]
Y_data = data[:, -1]

In [53]:
from sklearn.model_selection import train_test_split
# train test splitt

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

number_of_nodes = 7*7
symbol_names = ['RED', 'BLUE', 'EMPTY']

In [54]:
graphs_train = Graphs(X_train.shape[0],symbol_names=symbol_names, hypervector_size=128, hypervector_bits=2)

In [55]:
for graph_id in range(X_train.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes)


graphs_train.prepare_node_configuration()

# Add nodes to each graph
for graph_id in range(X_train.shape[0]):
    for node_id in range(number_of_nodes):
        nr_neighbours = len(getConnections(7, node_id))
        graphs_train.add_graph_node(graph_id, node_id, nr_neighbours)  # The '0' can be used as a placeholder

graphs_train.prepare_edge_configuration()



In [56]:

# Add edges to each graph
for graph_id in range(X_train.shape[0]):
    for node_id in range(number_of_nodes):
        # Get the list of adjacent nodes for the current node_id
        neighbors = getConnections(7,node_id) # add_graph_node_edge(graph_id, node_id, destination_node_id, edge_type)
        edge_type = 0
        for neighbor_id in neighbors:
            graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)

In [57]:
# assign features to the nodes
for graph_id in range(X_train.shape[0]):
    for node_id in range(number_of_nodes):
        # Get the value of the current node
        node_value = X_train[graph_id, node_id]
        # Assign the value to the node
        if node_value == 1:
            graphs_train.add_graph_node_feature(graph_id, node_id, 'RED')
        elif node_value == 2:
            graphs_train.add_graph_node_feature(graph_id, node_id, 'BLUE')
        else:
            graphs_train.add_graph_node_feature(graph_id, node_id, 'EMPTY')
       

In [58]:
# pickle dump the graphs_train object
import pickle
with open('graphs_train.pkl', 'wb') as f:
    pickle.dump(graphs_train, f)
    


In [59]:
import pickle
# load the graphs_train object
with open('graphs_train.pkl', 'rb') as f:
    graphs_train = pickle.load(f)

In [60]:
graphs_train.encode()

In [61]:
# create the test graphs object
graphs_test = Graphs(X_test.shape[0],symbol_names=symbol_names, hypervector_size=128, hypervector_bits=2)


In [62]:

for graph_id in range(X_test.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes)


graphs_test.prepare_node_configuration()

# Add nodes to each graph
for graph_id in range(X_test.shape[0]):
    for node_id in range(number_of_nodes):
        nr_neighbours = len(getConnections(7, node_id))
        graphs_test.add_graph_node(graph_id, node_id, nr_neighbours)  # The '0' can be used as placeholder

graphs_test.prepare_edge_configuration()

In [63]:
# Add edges to each graph

for graph_id in range(X_test.shape[0]):
    for node_id in range(number_of_nodes):
        # Get the list of adjacent nodes for the current node_id
        neighbors = getConnections(7,node_id) # add_graph_node_edge(graph_id, node_id, destination_node_id, edge_type)
        edge_type = 0
        for neighbor_id in neighbors:
            graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)

In [64]:

# assign features to the nodes

for graph_id in range(X_test.shape[0]):
    for node_id in range(number_of_nodes):
        # Get the value of the current node
        node_value = X_test[graph_id, node_id]
        # Assign the value to the node
        if node_value == 1:
            graphs_test.add_graph_node_feature(graph_id, node_id, 'RED')
        elif node_value == 2:
            graphs_test.add_graph_node_feature(graph_id, node_id, 'BLUE')
        else:
            graphs_test.add_graph_node_feature(graph_id, node_id, 'EMPTY')

In [65]:
# pickle dump the graphs_test object
import pickle
with open('graphs_test.pkl', 'wb') as f:
    pickle.dump(graphs_test, f)


In [66]:

# load the graphs_test object
with open('graphs_test.pkl', 'rb') as f:
    graphs_test = pickle.load(f)
    

In [67]:
graphs_test.encode()

In [68]:
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine

In [80]:
number_of_nodes = 7*7
number_of_clauses = 20000
T = 25000
s = 10.0
depth = 3
message_size = 256
message_bits = 2
max_included_literals = 32
number_of_states = 200
epochs = 250

In [81]:
tm = MultiClassGraphTsetlinMachine(number_of_clauses, T, s, depth=depth, message_size = message_size, message_bits = message_bits, max_included_literals=max_included_literals)


Initialization of sparse structure.


In [82]:
epochs



250

In [ ]:
# train the model
for i in range(epochs):
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)

    result_test = 100 * (tm.predict(graphs_test) == Y_test).mean()
    result_train = 100 * (tm.predict(graphs_train) == Y_train).mean()
    print("#%d Testing Accuracy: %.2f%% Training Accuracy: %.2f%%" % (i+1, result_test, result_train))

array([1504977930, 2724073097, 1059637269, ...,          0,          0,
                0], dtype=uint32)